# Task 3: Building the RAG Core Logic and Evaluation

In [3]:
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import os
import pickle
import sys
sys.path.append(os.path.abspath("../src"))
import warnings
warnings.filterwarnings('ignore')

In [4]:
# %%
os.chdir("..")  # Go up a directory
#print(os.getcwd())

In [5]:
from RAG_Pipeline import *

In [6]:
if __name__ == "__main__":
    question = "What are common complaints about credit card interest rates?"
    result = RAG_Pipeline(question)

    print(" Question:", result["question"])
    print("\n Retrieved Sources (1-2):")
    for source in result["retrieved_sources"]:
        print("-", source['product'], ":", source['original_text'][:200], "...\n")
    print(" Answer:\n", result["answer"])

NameError: name 'RAG_Pipeline' is not defined